# LangGraph with Reflexion: Self-Correcting Agents

This notebook illustrates the concept of langgraph with reflexion: self-correcting agents using LLMs.

# LangGraph with Reflexion: Self-Correcting Agents

This notebook demonstrates how to extend a LangGraph with self-reflection and retry capabilities, inspired by the Reflexion agent architecture.

In [1]:
# Install necessary libraries (if not already installed)
# !pip install langgraph langchain openai


In [2]:
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI

# Create LLM instance
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

ModuleNotFoundError: No module named 'langgraph'

## Define nodes for planning, execution, and reflection
- The `planner` creates a task.
- The `executor` attempts to solve it.
- The `reflector` decides whether to retry or stop.

In [ ]:
initial_state = {
    "goal": "Translate 'apple' to Spanish and check if it is a fruit",
    "attempts": 0,
    "max_attempts": 2
}

In [ ]:
# Planner: creates task steps
def planner(state):
    return {
        **state,
        "task": f"{state['goal']}"
    }

In [ ]:
# Executor: solves the task
def executor(state):
    response = llm.predict(state['task'])
    return {
        **state,
        "response": response
    }

In [ ]:
# Reflector: evaluates and decides whether to retry
def reflector(state):
    review_prompt = f"Was this response correct? '{state['response']}'"
    review = llm.predict(review_prompt)
    if "no" in review.lower() and state["attempts"] < state["max_attempts"]:
        print("Reflector: retrying...")
        return {
            **state,
            "attempts": state["attempts"] + 1,
            "retry": True
        }
    else:
        print("Reflector: done.")
        return {
            **state,
            "retry": False
        }

## Build the LangGraph with a loop from reflection back to execution

In [ ]:
# Build graph
workflow = StateGraph()
workflow.add_node("planner", planner)
workflow.add_node("executor", executor)
workflow.add_node("reflector", reflector)

workflow.set_entry_point("planner")
workflow.add_edge("planner", "executor")
workflow.add_edge("executor", "reflector")
workflow.add_conditional_edges("reflector", {
    "retry": "executor",
    "default": END
})

app = workflow.compile()

## Run the self-correcting agent

In [ ]:
app.invoke(initial_state)